In [1]:
without_punct_model_name = "thenlpresearcher/iitb-en-indic-without-punct"
with_punct_model_name = "thenlpresearcher/iitb-en-indic-only-punct"
combined_punct_model_name = "thenlpresearcher/iitb_en_indic_robust_punctuation_model"

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("thenlpresearcher/iitb_marathi_punct_variants")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["src"]
ref_gt     = raw_datasets['test']["tgt"]

In [ ]:
import re

def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        # Remove the prefix if present
        if t.startswith(prefix):
            t = t[len(prefix):].strip()
        # Normalize repeated punctuation
        t = re.sub(r'\.{2,}', '.', t)      # multiple dots → single dot
        t = re.sub(r'\?{2,}', '?', t)      # multiple question marks → single
        t = re.sub(r'-{2,}', '-', t)       # multiple dashes → single dash
        t = re.sub(r'_+', '_', t)          # multiple underscores → single underscore
        # Strip any trailing punctuation or spaces
        t = re.sub(r'^[\s\-\._]+|[\s\-\._]+$', '', t)
        ans.append(t)
    return ans

### Original

In [ ]:
model_name = "ai4bharat/indictrans2-en-indic-dist-200M"

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

In [ ]:
translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

In [ ]:
mode = "ai4bharat/original"

In [ ]:
import os
os.getcwd()

In [ ]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

### Model Fine-tuned on data without punctuation

In [ ]:
model_name = without_punct_model_name

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

In [ ]:
translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

In [ ]:
mode = "ai4bharat/without"

In [ ]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

## With punct model

In [ ]:
model_name = with_punct_model_name

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

In [ ]:
translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

In [ ]:
mode = "ai4bharat/with"

In [ ]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

## Combined

In [ ]:
model_name = combined_punct_model_name

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

In [ ]:
translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

In [ ]:
mode = "ai4bharat/combined"

In [ ]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")